In [ ]:
# Import packages
import cvxpy as cp
import gurobipy
import numpy as np
import pandas as pd
import os
import plotly.graph_objects as go
import matplotlib.pyplot as plt
from tabulate import tabulate

#functions necessary for UC setup 
from functions import (
    setup_optimization_problem_for_ap1000,
    update_kinf_and_deadtime,
    process_results,
    build_binary_var_table,
    compute_curtailment
)


In [ ]:
"""[DO NOT CHANGE THIS]"""
#This cell is tagged as a 'parameter' so that papermill can override the values and run this notebook

n = 0
nuclear_unit = 0
commit_at_len_T= {}
D_points={}
remaining_downtime = {} 
kinfTable_udt= {}
first_run = True
model_name= ''
load_percentage = 0
PHS_percentage = 0
PHS_duration= 0
net_cap = 0
horizon = 0
pmin = 0
VRE_percentage = 0
cost_frac=0
mAP1000=0
mAP300=0
mAP100=0

Wind_Cap=0
Solar_Cap=0
Nuclear_Cap=0
PHS_Cap=0
refuel_span=0

In [2]:
"""IMPORT DATA AND ROLLING DATAFRAMES"""

# Convert passed dictionaries from the last run to DataFrames
remaining_downtime = pd.DataFrame.from_dict(remaining_downtime)
kinfTable_udt      = pd.DataFrame.from_dict(kinfTable_udt)
commit_at_len_T    = pd.DataFrame.from_dict(commit_at_len_T)
D_points           = pd.DataFrame.from_dict(D_points)

# Current path
path = os.getcwd()

# ------------------------
# Data import
gen_info     = pd.read_csv(path + "/data/Generators_data_nuclear.csv")          # Generator info and cost assumptions
fuels        = pd.read_csv(path + "/data/Fuels_data.csv")                       # Fuel costs
gen_variable = pd.read_csv(path + "/data/Generators_variability_nuclear.csv")   # CF for all generators, includes geothermal
load_yearly  = pd.read_csv(path + "/data/ERCOT_south_load_wind_2021_2023.csv")
solar_wind_variablity_yearly = pd.read_csv(path + "/data/wind_solar_variability_ERCOT.csv")

# Deadtime and Pmin from Mathematica numerics
deadtime_matrix_ap1000 = pd.read_csv(path + "/data/resultmatrix_ap1000.csv")
deadtime_matrix_ap300  = pd.read_csv(path + "/data/resultmatrix_ap300.csv")
MaxReacTableAP1000     = pd.read_csv(path + "/data/reacvspminAP1000.csv")
MaxReacTableAP300      = pd.read_csv(path + "/data/reacvspminAP300.csv")

# ------------------------
# Select the correct hour indices for the current optimization block
T_period = range(n * 72 + 1, (n + 1) * 72 + 1)   # 3 days

# Load for T_period
loads = (
    load_yearly[load_yearly['Hour'].isin(T_period)]
    .reset_index(drop=True)
)
loads['Hour'] = range(1, 73)

# Isolate wind and solar hourly CF factors for T_period
wind_solar_T_period = (
    solar_wind_variablity_yearly[solar_wind_variablity_yearly['Hour'].isin(T_period)]
    .reset_index(drop=True)
)
hour_index = wind_solar_T_period["Hour"]  # Original time periods for later use in building transposed_df

# Replace wind and solar hourly CF factors in gen_variable for the current block
gen_variable["WEC_SDGE_onshore_wind_turbine_1.0"] = wind_solar_T_period["Wind"]
gen_variable["WEC_SDGE_solar_photovoltaic_1.0"]   = wind_solar_T_period["Solar"]

In [ ]:
"""Initial processing of gen_df"""
# (1) Rename all columns to lowercase (by convention)
for data in [gen_info, fuels, loads, gen_variable]:
    data.columns = data.columns.str.lower()

# (2) Keep only the first 26 columns in gen_info (relevant to the UC model)
gen_info = gen_info.iloc[:, 0:26]

# (3) Merge in fuel costs and add to generator dataframe
gen_df = (
    gen_info
    .merge(fuels, on="fuel", how="outer")
    .sort_values(by="r_id")
    .reset_index(drop=True)
)
gen_df.rename({'cost_per_mmbtu': 'fuel_cost'}, axis=1, inplace=True)

# (4) Add "is_variable" flag for variable generation sources (wind, solar)
gen_df["is_variable"] = False
gen_df.loc[
    gen_df["resource"].isin(["onshore_wind_turbine", "solar_photovoltaic"]),
    "is_variable"
] = True

# (5) Create full generator name: region + resource + cluster
gen_df["gen_full"] = (
    gen_df["region"] + "_" +
    gen_df["resource"] + "_" +
    gen_df["cluster"].astype(str)
)
gen_df["gen_full"] = gen_df["gen_full"].str.lower()

#take all capacities from CEM
gen_df.loc[gen_df['resource'] == 'onshore_wind_turbine', 'existing_cap_mw'] = int(Wind_Cap)
gen_df.loc[gen_df['resource'] == 'solar_photovoltaic',   'existing_cap_mw'] = int(Solar_Cap)
gen_df.loc[gen_df['resource'] == 'ap1000',   'existing_cap_mw'] = int(Nuclear_Cap)
gen_df.loc[gen_df['resource'] == 'hydroelectric_pumped_storage',   'existing_cap_mw'] = int(PHS_Cap)

# (6) Remove generators with no existing capacity
gen_df = gen_df[gen_df["existing_cap_mw"] > 0].reset_index(drop=True)

# Step 1: Filter relevant resources, but exclude all existing 'ap300'
keep_resources = ['hydroelectric_pumped_storage', 'onshore_wind_turbine', 'solar_photovoltaic']
gen_df_filtered = gen_df[gen_df['resource'].isin(keep_resources)].copy()

# Step 2: Get one representative 'ap300' row (you choose how – first, template, etc.)
template_ap1000_row = gen_df[gen_df['resource'] == 'ap1000'].iloc[0].copy()

# Step 3: Count reactors and reassign
cap = gen_df.loc[gen_df['resource'] == 'ap1000', 'existing_cap_mw'].values[0]
repeat_count = int(cap // 1000)

# Step 4: Repeat the ap300 row
ap1000_replicated = pd.DataFrame([template_ap1000_row] * repeat_count)

# Step 5: Combine
gen_df = pd.concat([gen_df_filtered, ap1000_replicated], ignore_index=True)
gen_df = gen_df.assign(existing_cap_mw = np.where(gen_df["resource"]=="ap1000", gen_df["existing_cap_mw"]/repeat_count, gen_df["existing_cap_mw"]))

#rename 'r_id' column depending on how many generators are left
for ii in range(gen_df.shape[0]):
    gen_df['r_id'][ii]=ii+1
    
gen_df['r_id'] = gen_df['r_id'].astype(int)     #this gets built in UC.ipynb independently


# (7) Reshape variable generation CF dataframe from wide to long format
gen_variable_long = pd.melt(
    gen_variable,
    id_vars=["hour"],
    var_name='gen_full',
    value_name='cf'
)

"""Make capacity of nuclear r_ids ZERO that have reached refueling for refuel_span"""

terminate_loop_indicator=pd.read_csv(f'terminate_loop_indicator_{n}.csv', index_col=0)
# Apply shutdown effects to gen_df and kinfTable_udt
shutdown_idx = terminate_loop_indicator.loc[
    terminate_loop_indicator['termination_condition'] == 1, 'r_id'
]

# Set existing_cap_mw = 0 in gen_df
gen_df.loc[gen_df['r_id'].isin(shutdown_idx), 'existing_cap_mw'] = 0.0


"""P_min base start"""
if model_name == 'ap1000':
    if pmin >= 1:
        # Force absolute minimum for uranium units
        gen_df.loc[gen_df['fuel'] == 'uranium', 'min_power'] = pmin
    else:
        # Merge p0 and update: if p0 < pmin → keep pmin; else → use p0; if p0 is NaN → keep existing
        merged_df = gen_df.merge(kinfTable_udt[['r_id', 'p0']], on='r_id', how='left')
        merged_df['min_power'] = np.where(
            merged_df['p0'].notna(),
            np.where(merged_df['p0'] < pmin, pmin, merged_df['p0']),
            merged_df['min_power']
        )
        merged_df.drop(columns='p0', inplace=True)
        gen_df = merged_df

G_nuclear_critical=[]

#we keep startup and shutdown cost same assuming similar startup/shutdown peocesses. We only change magnitude by multiplying with start_cost_frac
gen_df.loc[gen_df['fuel'] == 'uranium', 'start_cost_per_mw'] *= cost_frac
gen_df.loc[gen_df['fuel'] == 'uranium', 'shut_cost_per_mw'] *= cost_frac

In [ ]:
# """"""""""""""""""""""""""""""""""""""""""Set up and solve the problem""""""""""""""""""""""""""""""""""""""""""
#based on model_name use the appropriate function to set up optimization
if model_name == 'ap1000':
    result = setup_optimization_problem_for_ap1000(gen_df, loads, gen_variable_long, deadtime_matrix_ap1000,
                                          deadtime_matrix_ap300, D_points, first_run, remaining_downtime,\
                                                          commit_at_len_T, PHS_duration, G_nuclear_critical)

    
# Solve the problem
result['prob'].solve(
    solver=cp.GUROBI,
    verbose=True,
    MIPGap=0.001,               # 0.1% gap target (tight but achievable)
    TimeLimit=1000,             # Time limit in seconds (e.g., ~30 min)
    Method=2,                   # Barrier method for root LP
    Crossover=0,                # Skip crossover after barrier
    Presolve=2,                 # Aggressive presolve
    MIPFocus=1,                 # Focus on improving lower bound
    Cuts=2,                     # Aggressive cut generation
    Heuristics=0.1,             # Light heuristic use
    LogFile="gurobi.log"        # Save solver log to file
)


In [ ]:
"""[SAVE RESULTS] BUILD ALL DATAFRAME IN THE DESIRED FORMAT"""

#generation hourly per generator
gen_result_df = pd.DataFrame(np.row_stack(result['GEN'].value))
gen_result_df.index.name = 'generator'

#commit hourly per generator
commit_result_df = pd.DataFrame(np.row_stack(result['COMMIT'].value))
commit_result_df.index.name = 'generator'

#downtime hourly per generator that needs extending
dynamic_downtime_df = pd.DataFrame(np.row_stack(result['dynamic_downtime'].value))
dynamic_downtime_df.index.name = 'generator'


#shutdowns hourly per generator
shut_result_df = pd.DataFrame(np.row_stack(result['SHUT'].value))
shut_result_df.index.name = 'generator'

#startups hourly per generator
start_result_df = pd.DataFrame(np.row_stack(result['START'].value))
start_result_df.index.name = 'generator'

#net objective value
cost = result['prob'].value

# # Calculate curtailment = available wind and/or solar output that had to be wasted due to operating constraints
gen_result_df_long = gen_result_df.copy(deep=True)
gen_result_df_long["r_id"] = gen_df["r_id"]
gen_result_df_long = pd.melt(gen_result_df_long, id_vars=["r_id"], var_name='hour', value_name='gen')

#compute curtailment
curtail = compute_curtailment(
    gen_result_df=gen_result_df,
    gen_df=gen_df,
    result=result,
    hour_from_columns=True,     # or False with hour_index=...
    wind_tag="wec_sdge_onshore_wind_turbine_1.0",
    solar_tag="wec_sdge_solar_photovoltaic_1.0",
    wind_resource_name="onshore_wind_turbine",
    solar_resource_name="solar_photovoltaic",
    floor_zero=True
)

#PHS Results
gen_phs = gen_result_df_long[gen_result_df_long["r_id"] == 1].reset_index(drop=True)
gen_phs["charge"] = result["CHARGE"].value
gen_phs["discharge"] = result["DISCHARGE"].value
gen_phs["soc"]=result["SOC"].value

START  = build_binary_var_table(start_result_df, hour_index, "START", n)
SHUT   = build_binary_var_table(shut_result_df, hour_index, "SHUT", n)
GEN    = build_binary_var_table(gen_result_df, hour_index, "GEN", n)
COMMIT = build_binary_var_table(commit_result_df, hour_index, "COMMIT", n)

In [ ]:
#save commit results for analysis
commit_result = pd.DataFrame(np.row_stack(result['COMMIT'].value))

#Commit result at hour len(T)
commit_at_len_T = pd.DataFrame()
commit_at_len_T['commitment'] = commit_result.iloc[:, -1].round().astype(int)
commit_at_len_T['generator'] = gen_df['r_id'].astype(int)
commit_at_len_T['termination_condition'] = 0

# Calculate the potential soc_transfer value
potential_soc_transfer = gen_phs["soc"].iloc[-1] + (gen_phs["charge"].iloc[-1] * 0.84) - (gen_phs["discharge"].iloc[-1] / 0.84)

# Ensure soc_transfer is not negative
commit_at_len_T['soc_transfer'] = max(0, potential_soc_transfer)


#counts the total number of shutdowns each run
shut_count = (result['SHUT'].value==1).sum().sum()
start_count = (result['START'].value==1).sum().sum()
shut_df = pd.DataFrame({'n':[n],'shut_count': [shut_count],\
                        'start_count': [start_count],
                        'obj_value':[result['UCcost'].value]})

G_thermal = gen_df[gen_df["up_time"] > 0]["r_id"].tolist() #thermal generators
remaining_downtime = pd.DataFrame(columns=['generator','remaining_downtime'])
remaining_downtime['generator'] = G_thermal

# Extract remaining downtime for next run
T = loads["hour"]
for i in G_thermal:
    for t in range(len(T)):
        if shut_result_df.loc[i-1, t] == 1:
            downtime_start = t
            downtime_duration = dynamic_downtime_df.loc[i-1, t]  # Assuming dynamic_downtime_df is a DataFrame
            remaining_hours_in_window = len(T) - downtime_start  # Time remaining in this optimization window
            # Calculate remaining downtime that will carry over to the next optimization period
            remaining_downtime_for_next_period = max(0, downtime_duration - remaining_hours_in_window)
            remaining_downtime.loc[remaining_downtime['generator'] == i, \
            'remaining_downtime'] = int(remaining_downtime_for_next_period)


# Fill NaN values with 0 (or any other value you want)
remaining_downtime.fillna(0, inplace=True)
remaining_downtime.reset_index(drop=True, inplace=True)


remaining_downtime.to_csv(f'remaining_downtime_{n+1}.csv')
shut_df.to_csv(f'shut_df_{n}.csv')

In [ ]:
#aggregate results for post process and nect run
gen_result_df_grouped = gen_result_df.copy(deep=True)
gen_result_df_grouped["resource"] = gen_df["resource"]
gen_result_df_grouped = gen_result_df_grouped.groupby('resource', as_index=False).sum()

#inputs to process_results
transposed_df = process_results(
    gen_result_df_grouped, gen_phs, loads, result, curtail,
    model_name, hour_index, n
)

In [ ]:
#degrades k_eff values, updates deadtime for next runs and updates termiantion indicator

kinfTable_udt, D_points, terminate_loop_indicator, commit_at_len_T = update_kinf_and_deadtime(
    n, kinfTable_udt, gen_result_df, gen_df,
    MaxReacTableAP1000, MaxReacTableAP300,
    deadtime_matrix_ap1000, deadtime_matrix_ap300,
    terminate_loop_indicator, commit_at_len_T, D_points,
    mAP1000, mAP300, T, pmin, refuel_span
)